In [2]:
import sys
import glob
import numpy as np
import collections
import matplotlib.pyplot as plt

In [3]:
def makeReplicationHistForRunMode(runMode, percentileThresh, tsOrFet, numReps=11, winSize=10000):

    assert tsOrFet.lower() in ["ts","fet"]

    allScoreCompFileNames = glob.glob(f"/pine/scr/d/s/dschride/data/timeSeriesSweeps/compareToFETOut/rep*ScoreComp*_{runMode}.txt")

    scoreIndex = 2 if tsOrFet == "ts" else 3

    scores = []
    for cfn in allScoreCompFileNames:
        #sys.stderr.write(f'reading {cfn}\n')
        with open(cfn, 'rt') as cf:
            for line in cf:
                scores.append(float(line.strip().split()[scoreIndex]))
    threshold = np.percentile(scores, percentileThresh*100)
    sys.stderr.write(f'chosen threshold for {runMode}: ({percentileThresh*100}th percentile): {threshold}\n')

    repCounts = {}
    for rep in range(1, numReps):
        sys.stderr.write(f'processing rep {rep}\n')
        fn = f"/pine/scr/d/s/dschride/data/timeSeriesSweeps/replicationOfTopHits{'' if tsOrFet == 'ts' else 'FET'}/rep{rep}_{runMode}_repComp.txt"
        with open(fn, 'rt') as f:
            for line in f:
                line = line.strip()
                splitLine = line.split("\t")
                chrom, pos, tsScore, fetScore, freqVel, otherTsScoreStr, otherFetScoreStr, otherFreqVelsStr = splitLine
                pos = int(pos)
                pos = pos-(pos % winSize)

                if tsOrFet == "ts":
                    otherScores = [float(x) for x in otherTsScoreStr.split("|")]
                else:
                    otherScores = [float(x) for x in otherFetScoreStr.split("|")]

                replicationVector = [1 if x > threshold else 0 for x in otherScores]
                if (chrom, pos) in repCounts:
                    repCounts[(chrom, pos)] = max(sum(replicationVector), repCounts[(chrom, pos)])
                else:
                    repCounts[(chrom, pos)] = sum(replicationVector)
                    
    counts = collections.Counter(repCounts.values())
    outx, outy = [], []
    for i in range(0, numReps):
        print(tsOrFet, runMode, i, counts[i]/sum(counts.values()), counts[i])
        outy.append(counts[i]/sum(counts.values()))
        outx.append(i)
    return outx, outy

In [4]:
def plotBars(tsX, tsY, fetX, fetY, title):
    """code modified from
    https://matplotlib.org/stable/gallery/lines_bars_and_markers/barchart.html"""
    
    assert tsX == fetX
    
    width = 0.35
    fig, ax = plt.subplots()
    rects1 = ax.bar(np.array(tsX) - width/2, tsY, width, label='TimeSweeper')
    rects2 = ax.bar(np.array(fetX) + width/2, fetY, width, label='FET')

    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel('Fraction of hits')
    ax.set_xlabel('Number of times a hit was replicated')
    ax.set_title(title)
    ax.legend()

    fig.tight_layout()

    plt.show()

In [5]:
runMode = "unif_vel_0_thresh"
percentileThresh = 0.99
tsX, tsY = makeReplicationHistForRunMode(runMode, percentileThresh, "ts", winSize=10000)
print(tsX, tsY)
fetX, fetY = makeReplicationHistForRunMode(runMode, percentileThresh, "fet", winSize=10000)
plotBars(tsX, tsY, fetX, fetY, "Replication Comparison: 10 kb windows")
print(fetX, fetY)
plt.savefig("S34_Replication_Hist.pdf")

IndexError: cannot do a non-empty take from an empty axes.

In [6]:
runMode = "unif_vel_0_thresh"
percentileThresh = 0.99
tsX, tsY = makeReplicationHistForRunMode(runMode, percentileThresh, "ts", winSize=100000)
fetX, fetY = makeReplicationHistForRunMode(runMode, percentileThresh, "fet", winSize=100000)
plotBars(tsX, tsY, fetX, fetY, "replication comparison: 100 kb windows")

IndexError: cannot do a non-empty take from an empty axes.